# Lennard-Jones Potential Playground

Interactive exploration of the Lennard-Jones 12-6 potential:

$$V(r) = 4\varepsilon \left[\left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6\right]$$

where:
- $V(r)$: potential energy
- $r$: interparticle distance
- $\varepsilon$: depth of potential well (strength of attraction)
- $\sigma$: finite distance where $V(r) = 0$

---

## Setup

Import necessary modules and configure the notebook.

In [ ]:
# Add src directory to path
import sys
from pathlib import Path

# Add parent directory's src to path
src_path = Path.cwd().parent / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import LJ modules
from model import (
    lj_potential, 
    lj_force, 
    lj_equilibrium, 
    generate_lj_curve,
    morse_potential,
    reduced_lj_potential
)
from utils import format_energy, format_distance

# Import plotting libraries
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# For interactive widgets
from ipywidgets import interact, FloatSlider, Checkbox, HBox, VBox, Output

print("✓ All modules loaded successfully!")

## 1. Basic LJ Potential Plot

Let's start with a simple plot of the Lennard-Jones potential.

In [ ]:
# Set default parameters
epsilon = 1.0  # kJ/mol
sigma = 3.5    # Angstrom

# Generate LJ curve
r, V = generate_lj_curve(epsilon, sigma)
r_eq, V_eq = lj_equilibrium(epsilon, sigma)

# Create plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(r, V, 'b-', linewidth=2, label='LJ Potential')
ax.axhline(y=0, color='k', linestyle='--', linewidth=0.8, alpha=0.5, label='V = 0')
ax.plot(r_eq, V_eq, 'ro', markersize=10, 
        label=f'Minimum: r={r_eq:.2f} Å, V={V_eq:.2f} kJ/mol')

ax.set_xlabel('Distance r (Å)', fontsize=12)
ax.set_ylabel('Potential V(r) (kJ/mol)', fontsize=12)
ax.set_title(f'Lennard-Jones Potential (ε={epsilon} kJ/mol, σ={sigma} Å)', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=10)

plt.tight_layout()
plt.show()

print(f"Equilibrium distance: {format_distance(r_eq)}")
print(f"Minimum energy: {format_energy(V_eq)}")
print(f"r_min/σ ratio: {r_eq/sigma:.4f} (theoretical: {2**(1/6):.4f})")

## 2. Potential and Force Together

The force is the negative derivative of the potential: $F(r) = -\frac{dV}{dr}$

In [ ]:
# Generate data
r, V = generate_lj_curve(epsilon, sigma)
F = lj_force(r, epsilon, sigma)
r_eq, V_eq = lj_equilibrium(epsilon, sigma)

# Create figure with two y-axes
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot potential on left axis
color1 = 'tab:blue'
ax1.set_xlabel('Distance r (Å)', fontsize=12)
ax1.set_ylabel('Potential V(r) (kJ/mol)', color=color1, fontsize=12)
ax1.plot(r, V, color=color1, linewidth=2, label='LJ Potential')
ax1.axhline(y=0, color='gray', linestyle='--', linewidth=0.8, alpha=0.5)
ax1.plot(r_eq, V_eq, 'ro', markersize=10, label='Equilibrium')
ax1.tick_params(axis='y', labelcolor=color1)
ax1.grid(True, alpha=0.3)

# Plot force on right axis
ax2 = ax1.twinx()
color2 = 'tab:red'
ax2.set_ylabel('Force F(r) (kJ/mol/Å)', color=color2, fontsize=12)
ax2.plot(r, F, color=color2, linewidth=2, linestyle='--', alpha=0.7, label='LJ Force')
ax2.axhline(y=0, color='red', linestyle=':', linewidth=0.8, alpha=0.3)
ax2.tick_params(axis='y', labelcolor=color2)

# Add title and legends
plt.title(f'LJ Potential and Force (ε={epsilon} kJ/mol, σ={sigma} Å)', 
          fontsize=14, fontweight='bold')

# Combine legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=10)

fig.tight_layout()
plt.show()

print("Force interpretation:")
print("  • Positive force (F > 0): Repulsive - particles push apart")
print("  • Negative force (F < 0): Attractive - particles pull together")
print("  • Force = 0 at equilibrium distance")

## 3. Interactive Exploration with Matplotlib

Use sliders to interactively adjust ε and σ parameters.

In [ ]:
%matplotlib widget

def plot_lj_interactive(epsilon, sigma, show_force, show_morse):
    """Interactive plotting function."""
    # Clear previous plot
    plt.clf()
    
    # Generate data
    r, V = generate_lj_curve(epsilon, sigma)
    r_eq, V_eq = lj_equilibrium(epsilon, sigma)
    
    if show_force:
        # Create figure with two y-axes
        fig, ax1 = plt.subplots(figsize=(12, 6))
        
        # Plot potential
        ax1.plot(r, V, 'b-', linewidth=2, label='LJ Potential')
        ax1.axhline(y=0, color='gray', linestyle='--', linewidth=0.8, alpha=0.5)
        ax1.plot(r_eq, V_eq, 'ro', markersize=8, label='Equilibrium')
        ax1.set_xlabel('Distance r (Å)', fontsize=11)
        ax1.set_ylabel('Potential (kJ/mol)', color='b', fontsize=11)
        ax1.tick_params(axis='y', labelcolor='b')
        ax1.grid(True, alpha=0.3)
        
        # Plot force
        ax2 = ax1.twinx()
        F = lj_force(r, epsilon, sigma)
        ax2.plot(r, F, 'r--', linewidth=2, alpha=0.7, label='LJ Force')
        ax2.axhline(y=0, color='red', linestyle=':', linewidth=0.8, alpha=0.3)
        ax2.set_ylabel('Force (kJ/mol/Å)', color='r', fontsize=11)
        ax2.tick_params(axis='y', labelcolor='r')
        
        # Combine legends
        lines1, labels1 = ax1.get_legend_handles_labels()
        lines2, labels2 = ax2.get_legend_handles_labels()
        ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right', fontsize=9)
    else:
        # Simple plot
        fig, ax1 = plt.subplots(figsize=(10, 6))
        ax1.plot(r, V, 'b-', linewidth=2, label='LJ Potential')
        ax1.axhline(y=0, color='gray', linestyle='--', linewidth=0.8, alpha=0.5)
        ax1.plot(r_eq, V_eq, 'ro', markersize=8, label='Equilibrium')
        ax1.set_xlabel('Distance r (Å)', fontsize=11)
        ax1.set_ylabel('Potential V(r) (kJ/mol)', fontsize=11)
        ax1.grid(True, alpha=0.3)
        ax1.legend(fontsize=9)
    
    # Add Morse potential if requested
    if show_morse:
        D_e = epsilon
        r_e = r_eq
        a = np.sqrt(36 * epsilon / (sigma ** 2)) / 2
        V_morse = morse_potential(r, D_e, a, r_e)
        ax1.plot(r, V_morse, 'g--', linewidth=2, alpha=0.7, label='Morse Potential')
        ax1.legend(loc='upper right', fontsize=9)
    
    plt.title(f'ε = {epsilon:.2f} kJ/mol, σ = {sigma:.2f} Å\n' +
              f'r_min = {r_eq:.2f} Å, V_min = {V_eq:.2f} kJ/mol',
              fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Create interactive widget
interact(
    plot_lj_interactive,
    epsilon=FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, 
                        description='ε (kJ/mol):'),
    sigma=FloatSlider(value=3.5, min=2.0, max=5.0, step=0.1, 
                      description='σ (Å):'),
    show_force=Checkbox(value=False, description='Show Force'),
    show_morse=Checkbox(value=False, description='Show Morse')
);

## 4. Interactive Plotly Visualization

Create a fully interactive Plotly plot with hover information.

In [ ]:
def create_plotly_lj(epsilon, sigma, show_force, show_morse):
    """Create interactive Plotly visualization."""
    # Generate data
    r, V = generate_lj_curve(epsilon, sigma, n_points=500)
    r_eq, V_eq = lj_equilibrium(epsilon, sigma)
    
    # Create figure
    if show_force:
        fig = make_subplots(specs=[[{"secondary_y": True}]])
    else:
        fig = go.Figure()
    
    # Add LJ potential
    if show_force:
        fig.add_trace(
            go.Scatter(x=r, y=V, mode='lines', name='LJ Potential',
                      line=dict(color='blue', width=3),
                      hovertemplate='r: %{x:.2f} Å<br>V: %{y:.2f} kJ/mol<extra></extra>'),
            secondary_y=False
        )
    else:
        fig.add_trace(
            go.Scatter(x=r, y=V, mode='lines', name='LJ Potential',
                      line=dict(color='blue', width=3),
                      hovertemplate='r: %{x:.2f} Å<br>V: %{y:.2f} kJ/mol<extra></extra>')
        )
    
    # Add equilibrium point
    if show_force:
        fig.add_trace(
            go.Scatter(x=[r_eq], y=[V_eq], mode='markers', name='Minimum',
                      marker=dict(color='red', size=12, symbol='circle'),
                      hovertemplate=f'Equilibrium<br>r: {r_eq:.2f} Å<br>V: {V_eq:.2f} kJ/mol<extra></extra>'),
            secondary_y=False
        )
    else:
        fig.add_trace(
            go.Scatter(x=[r_eq], y=[V_eq], mode='markers', name='Minimum',
                      marker=dict(color='red', size=12, symbol='circle'),
                      hovertemplate=f'Equilibrium<br>r: {r_eq:.2f} Å<br>V: {V_eq:.2f} kJ/mol<extra></extra>')
        )
    
    # Add V=0 reference line
    if show_force:
        fig.add_trace(
            go.Scatter(x=r, y=np.zeros_like(r), mode='lines',
                      line=dict(color='gray', width=1, dash='dash'),
                      showlegend=False, hoverinfo='skip'),
            secondary_y=False
        )
    else:
        fig.add_trace(
            go.Scatter(x=r, y=np.zeros_like(r), mode='lines',
                      line=dict(color='gray', width=1, dash='dash'),
                      showlegend=False, hoverinfo='skip')
        )
    
    # Add force curve if requested
    if show_force:
        F = lj_force(r, epsilon, sigma)
        fig.add_trace(
            go.Scatter(x=r, y=F, mode='lines', name='LJ Force',
                      line=dict(color='red', width=2, dash='dot'),
                      opacity=0.7,
                      hovertemplate='r: %{x:.2f} Å<br>F: %{y:.2f} kJ/mol/Å<extra></extra>'),
            secondary_y=True
        )
        
        # F=0 line
        fig.add_trace(
            go.Scatter(x=r, y=np.zeros_like(r), mode='lines',
                      line=dict(color='red', width=1, dash='dash'),
                      opacity=0.3, showlegend=False, hoverinfo='skip'),
            secondary_y=True
        )
    
    # Add Morse potential if requested
    if show_morse:
        D_e = epsilon
        r_e = r_eq
        a = np.sqrt(36 * epsilon / (sigma ** 2)) / 2
        V_morse = morse_potential(r, D_e, a, r_e)
        
        if show_force:
            fig.add_trace(
                go.Scatter(x=r, y=V_morse, mode='lines', name='Morse Potential',
                          line=dict(color='green', width=2, dash='dash'),
                          opacity=0.7,
                          hovertemplate='r: %{x:.2f} Å<br>V: %{y:.2f} kJ/mol<extra></extra>'),
                secondary_y=False
            )
        else:
            fig.add_trace(
                go.Scatter(x=r, y=V_morse, mode='lines', name='Morse Potential',
                          line=dict(color='green', width=2, dash='dash'),
                          opacity=0.7,
                          hovertemplate='r: %{x:.2f} Å<br>V: %{y:.2f} kJ/mol<extra></extra>')
            )
    
    # Update layout
    if show_force:
        fig.update_xaxes(title_text="Distance r (Å)", gridcolor='lightgray')
        fig.update_yaxes(title_text="Potential V(r) (kJ/mol)", secondary_y=False,
                        gridcolor='lightgray', titlefont=dict(color='blue'))
        fig.update_yaxes(title_text="Force F(r) (kJ/mol/Å)", secondary_y=True,
                        titlefont=dict(color='red'))
    else:
        fig.update_xaxes(title_text="Distance r (Å)", gridcolor='lightgray')
        fig.update_yaxes(title_text="Potential V(r) (kJ/mol)", gridcolor='lightgray')
    
    fig.update_layout(
        title=dict(
            text=f'Lennard-Jones Potential Explorer<br>' + 
                 f'<sub>ε = {epsilon:.2f} kJ/mol, σ = {sigma:.2f} Å | ' +
                 f'r_min = {r_eq:.2f} Å, V_min = {V_eq:.2f} kJ/mol</sub>',
            x=0.5,
            xanchor='center'
        ),
        height=650,
        hovermode='closest',
        template='plotly_white',
        legend=dict(x=0.98, y=0.98, xanchor='right', yanchor='top')
    )
    
    return fig

# Create interactive widget with Plotly
interact(
    create_plotly_lj,
    epsilon=FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1,
                        description='ε (kJ/mol):', continuous_update=False),
    sigma=FloatSlider(value=3.5, min=2.0, max=5.0, step=0.1,
                      description='σ (Å):', continuous_update=False),
    show_force=Checkbox(value=True, description='Show Force'),
    show_morse=Checkbox(value=False, description='Show Morse')
);

## 5. Reduced Units Analysis

Explore the universal form of the LJ potential using reduced units:
- $r^* = r/\sigma$ (reduced distance)
- $V^* = V/\varepsilon$ (reduced potential)

In [ ]:
# Generate reduced LJ curve
r_star = np.linspace(0.8, 3.0, 500)
V_star = reduced_lj_potential(r_star)

# Create plot
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(r_star, V_star, 'b-', linewidth=2, label='Reduced LJ Potential')
ax.axhline(y=0, color='k', linestyle='--', linewidth=0.8, alpha=0.5)
ax.axvline(x=1, color='g', linestyle=':', linewidth=1, alpha=0.5, label='r* = 1 (V* = 0)')
ax.axvline(x=2**(1/6), color='r', linestyle=':', linewidth=1, alpha=0.5, 
           label=f'r* = 2^(1/6) ≈ {2**(1/6):.3f} (minimum)')

# Mark minimum
r_star_min = 2**(1/6)
V_star_min = reduced_lj_potential(r_star_min)
ax.plot(r_star_min, V_star_min, 'ro', markersize=10, 
        label=f'Min: (r*={r_star_min:.3f}, V*={V_star_min:.3f})')

ax.set_xlabel('Reduced Distance r* = r/σ', fontsize=12)
ax.set_ylabel('Reduced Potential V* = V/ε', fontsize=12)
ax.set_title('Universal Lennard-Jones Potential (Reduced Units)', 
             fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=9)
ax.set_ylim(-1.5, 2)

plt.tight_layout()
plt.show()

print("Key Features of Reduced LJ Potential:")
print(f"  • Zero crossing at r* = 1 (corresponds to r = σ)")
print(f"  • Minimum at r* = 2^(1/6) ≈ {2**(1/6):.4f}")
print(f"  • Minimum value V* = -1 (corresponds to V = -ε)")
print(f"  • Universal shape - all LJ potentials look identical in reduced units!")

## 6. Comparing Different Parameter Sets

Visualize how different atoms/molecules with different LJ parameters behave.

In [ ]:
# Define LJ parameters for different systems (example values)
systems = {
    'Argon': {'epsilon': 0.997, 'sigma': 3.40, 'color': 'blue'},
    'Neon': {'epsilon': 0.314, 'sigma': 2.74, 'color': 'green'},
    'Xenon': {'epsilon': 1.77, 'sigma': 3.96, 'color': 'red'},
    'Generic': {'epsilon': 1.0, 'sigma': 3.5, 'color': 'purple'}
}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot all systems
for name, params in systems.items():
    r, V = generate_lj_curve(params['epsilon'], params['sigma'])
    r_eq, V_eq = lj_equilibrium(params['epsilon'], params['sigma'])
    
    # Normal units
    ax1.plot(r, V, linewidth=2, color=params['color'], label=name)
    ax1.plot(r_eq, V_eq, 'o', markersize=6, color=params['color'])
    
    # Reduced units
    r_star = r / params['sigma']
    V_star = V / params['epsilon']
    ax2.plot(r_star, V_star, linewidth=2, color=params['color'], 
             label=name, linestyle='--', alpha=0.7)

# Format left plot (normal units)
ax1.axhline(y=0, color='k', linestyle='--', linewidth=0.8, alpha=0.3)
ax1.set_xlabel('Distance r (Å)', fontsize=12)
ax1.set_ylabel('Potential V(r) (kJ/mol)', fontsize=12)
ax1.set_title('LJ Potentials - Physical Units', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=10)
ax1.set_ylim(-2, 3)

# Format right plot (reduced units)
ax2.axhline(y=0, color='k', linestyle='--', linewidth=0.8, alpha=0.3)
ax2.set_xlabel('Reduced Distance r* = r/σ', fontsize=12)
ax2.set_ylabel('Reduced Potential V* = V/ε', fontsize=12)
ax2.set_title('LJ Potentials - Reduced Units (Universal!)', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=10)
ax2.set_ylim(-1.5, 2)

plt.tight_layout()
plt.show()

print("\nComparison Summary:")
print("-" * 60)
print(f"{'System':<12} {'ε (kJ/mol)':<12} {'σ (Å)':<10} {'r_min (Å)':<12}")
print("-" * 60)
for name, params in systems.items():
    r_eq, _ = lj_equilibrium(params['epsilon'], params['sigma'])
    print(f"{name:<12} {params['epsilon']:<12.3f} {params['sigma']:<10.2f} {r_eq:<12.2f}")
print("-" * 60)

## 7. Export Data for MD Simulations

Export potential data to CSV for use in molecular dynamics simulations.

In [ ]:
from utils import save_potential_table

# Generate high-resolution data
epsilon = 1.0
sigma = 3.5
r, V = generate_lj_curve(epsilon, sigma, n_points=1000)
F = lj_force(r, epsilon, sigma)

# Save to CSV
output_file = '../examples/lj_potential_data.csv'
save_potential_table(r, V, output_file, include_force=True, F=F)

# Display first few rows
import pandas as pd
df = pd.read_csv(output_file)
print("\nFirst 10 rows of exported data:")
print(df.head(10))
print(f"\nTotal rows exported: {len(df)}")
print(f"File saved to: {output_file}")